In [149]:
using FastGaussQuadrature
using BenchmarkTools

"""
    function integrate_mc(f, xmin, xmax, num_points)

Integrates function f over the hyper-rectangle [xmin[1], xmax[1]] × [xmin[2], xmax[2]] × ... × [xmin[d], xmax[d]]
using Monte Carlo.
"""

function integrate_mc_3d(f::Function, xmin::Vector{Float64}, xmax::Vector{Float64}, num_points::Int)
    @assert length(xmin) == length(xmax)
    d = length(xmin) # the dimension
    X = rand(Float64, num_points, d)

    V = reduce(*, xmax-xmin)

    function eval(x)
        xx = x.* (xmax - xmin) + xmin
        return f( xx[1], xx[2], xx[3])
    end
    return sum( map( eval, eachrow(X))  ) * V/ num_points
end

function integrate_gauss_3d(f::Function, xmin::Vector{Float64}, xmax::Vector{Float64}, num_points::Int)
    ξ, w = gausslegendre( num_points )
    ξ = 0.5 .* ξ .+ 0.5
    x = (xmax[1] - xmin[1]).* ξ .+ xmin[1]
    y = (xmax[2] - xmin[2]).* ξ .+ xmin[2]
    z = (xmax[3] - xmin[3]).* ξ .+ xmin[3]
    
    sum = 0.
    for ix=1:num_points, iy=1:num_points, iz=1:num_points
        sum += f(x[ix], y[iy], z[iz]) * w[ix] * w[iy] * w[iz]
    end

    V = reduce(*, xmax-xmin)/8

    return sum *V
end


integrate_gauss_3d (generic function with 2 methods)

In [173]:
testf = (x,y,z)-> sqrt(sin(x - y) + cos(y-z)+2.1)
L = 11
# exact = 8π/L * sin(π/L)^2 

11

In [177]:
@benchmark integrate_mc_3d( testf, -π*ones(3)/L,  π*ones(3)/L, 100000 )

BenchmarkTools.Trial: 305 samples with 1 evaluation.
 Range (min … max):  14.486 ms … 24.294 ms  ┊ GC (min … max):  0.00% … 8.70%
 Time  (median):     16.690 ms              ┊ GC (median):    12.41%
 Time  (mean ± σ):   16.023 ms ±  1.303 ms  ┊ GC (mean ± σ):   7.81% ± 6.23%

   ▆▆                       █▄▆                                
  ▇██▅▃▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅███▆▃▃▃▂▂▁▁▁▂▁▂▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▂▁▂ ▃
  14.5 ms         Histogram: frequency by time        19.4 ms <

 Memory estimate: 25.94 MiB, allocs estimate: 300012.

In [178]:
@benchmark integrate_gauss_3d( testf, -π*ones(3)/L, π*ones(3)/L, 11 )

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.875 μs …  17.223 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     25.417 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   40.199 μs ± 250.919 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▂▁           ▁                                             ▁
  ███████████▇█▇███▇▇▆▇▆▆▆▆▆▇▆▆▅▆▅▅▅▅▅▅▄▅▅▃▅▄▄▃▄▄▄▄▄▄▄▃▃▄▅▄▃▃▃ █
  22.9 μs       Histogram: log(frequency) by time       206 μs <

 Memory estimate: 2.48 KiB, allocs estimate: 25.